#2 Run
This file takes in the file written as a .csv from naive.ipynb. The output is used by compare.ipynb

In [6]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
import pandas as pd
import tensorflow as tf

In [7]:
def read_file(file):
    """Read a single file and return a dataframe"""
    return pd.read_csv(file, skipinitialspace=True)

In [8]:
df_options = read_file("data/benchmark_with_pred_naive.csv")

In [9]:
# Black-Scholes formula for call options
def d1(S,K,T,r,sigma):
    x1 = S.apply(lambda x : log(x)) - K.apply(lambda x : log(x))
    x2 = (r + ((sigma.apply(lambda x : x**2)) / 2)) * T
    x3 = sigma * T.apply(lambda x: sqrt(x))
    return  (x1 + x2) / x3

def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma) - sigma * T.apply(lambda x : sqrt(x))  

def bs_call(S,K,T,r,sigma):
    T = T/365
    r = r/100
    return S * d1(S,K,T,r,sigma).apply(lambda x : norm.cdf(x)) - K * (-r*T).apply(lambda x : exp(x)) * d2(S,K,T,r,sigma).apply(lambda x : norm.cdf(x))

In [10]:
df_options["BS"] = bs_call(df_options["Underlying_last"], df_options["Strike"], df_options["Ttl"], df_options["R"], df_options["Volatility"])


In [11]:
print(df_options.head())

   Quote_date  Underlying_last  Strike  Ttl  Volatility     R     Price  \
0  2021-01-08          3824.45  1000.0    3    0.186024  0.08  2826.250   
1  2021-01-08          3824.45  3860.0   49    0.186024  0.08    75.200   
2  2021-01-08          3824.45  3865.0   49    0.186024  0.08    72.955   
3  2021-01-08          3824.45  3870.0   49    0.186024  0.08    70.550   
4  2021-01-08          3824.45  3875.0   49    0.186024  0.08    67.495   

      Naive   Prediction           BS  
0  2803.855  2774.402600  2824.456575  
1    67.900    69.269806    87.819989  
2    65.210    66.548515    85.672920  
3    63.055    63.835953    83.563047  
4    60.500    61.140602    81.490186  


In [12]:
mse_bs = tf.keras.metrics.mean_squared_error(
    df_options["Price"], df_options["BS"]
)

In [13]:
print("MSE: ", mse_bs)
print("RMSE: ", sqrt(mse_bs))

MSE:  tf.Tensor(1430.597686425085, shape=(), dtype=float64)
RMSE:  37.82324267464498


In [14]:
# Write to file
df_options.to_csv('data/benchmark_with_Pred_Naive_and_BS.csv', encoding='utf-8', index=False)